<a href="https://colab.research.google.com/github/CAB314/RSA-cryptosystem/blob/main/RSAmaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSA maker

## Make dictionary

In [ ]:
def mkdic(s):
  cs = [c for c in s]
  dic_c_i = { ch:i for i,ch in enumerate(cs)}
  dic_i_c  = { i:ch for i,ch in enumerate(cs)}
  return dic_c_i, dic_i_c

This RSA cipher generator has default vocabrary (A-Z, a-z, "!", "?", ",", ".", " " ). 

$d$ is the vocabrary list (the type is string) like "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz!?,. "

You can create the vocabrary list(string) including any character.

In [ ]:
d = ""
for i in range(65, 65+26):
  d += chr(i)
for i in range(97, 97+26):
  d += chr(i)

d += chr(33)
d += chr(63)
d += chr(44)
d += chr(46)
d += " "
d

In [ ]:
dic_c_i, dic_i_c = mkdic(d)
print(dic_c_i)
print(dic_i_c)

## Define keys

Generate prime number $p$ and $q$ ($\text {vocabrary size}+5 < p, q <  10^{501}$)

In [ ]:
import sympy

p = sympy.randprime(len(dic_c_i)+5, 10**501)
q = sympy.randprime(len(dic_c_i)+5, 10**501)
while p == q:
  q = sympy.randprime(len(dic_c_i)+5, 10**501)

Generate public key $key_n, key_e$ and private key $key_p$.

In [ ]:


def gcd(a, b):
    while b:
        a, b = b, a % b
    return a
    
def key_pub(p, q, div = 3):
  key_n = p*q
  p_q_ = (p-1)*(q-1)
  key_e = 0
  for i in reversed(range(int(p_q_//div))):
    if (gcd(p_q_, i) == 1):
      key_e = i
      break
  return key_n, key_e

key_n, key_e= key_pub(p, q)

def key_pri(p, q, key_e):
  a = key_e
  b = (p-1)*(q-1)
  x1, y1, _ = sympy.gcdex(a,b)
  #a(x-x1) = -b(y-y1)
  #x-x1 = bm
  #y-y1 = -am
  m = 1
  x = b*m + x1
  while(x > 0):
    m-=1
    x = b*m + x1
  while(x <= 0):
    m+=1
    x = b*m + x1
  return x

key_p = int(key_pri(p, q, key_e))

print(key_n, key_e)
print(key_p)

## plaintext to int

In [ ]:
def plain_to_int(dic_c_i, text):
  return [dic_c_i[c] for c in text]
def int_to_plain(dic_i_c, nums):
  return [dic_i_c[c] for c in nums]

answer =  "" #Text that you want to send using RSA.
if answer == "":
  answer = input()
nums = plain_to_int(dic_c_i, answer)
text = int_to_plain(dic_i_c, nums)

print(nums)
print("".join(text))

## ciphertext

Answer numbers($nums$) to ciphertext($nums_d$)

In [ ]:
from multiprocessing import Process, Queue
def power_func(num,key_e,key_n, que, itr):
    bi = str(format(key_e,"b"))
    res = 1
    for i in range(len(bi)):
        res = (res*res) %key_n
        if bi[i] == "1":
            res = (res*num) %key_n
    que.put((itr, res))


que = Queue()
nums_d = nums.copy()
for i in range(len(nums)):
  num = nums[i]
  p = Process(target=power_func, args=(num,key_e,key_n, que, i))
  p.start()

values = sorted((que.get() for _ in range(len(nums))), key=lambda r: r[0])
for i, v in enumerate(values):
  nums_d[i] = v[1]

In [ ]:
print(nums_d)

# Decipher

Ciphertext numbers($nums_d$) to plaintext numbers($nums_{test}$)

In [ ]:
def power_func(num,key_e,key_n, que, itr):
    bi = str(format(key_e,"b"))#2進表現に
    res = 1
    for i in range(len(bi)):
        res = (res*res) %key_n
        if bi[i] == "1":
            res = (res*num) %key_n
    que.put((itr, res))

nums_test = nums_d.copy()
que = Queue()
for i in range(len(nums_d)):
  num = nums_d[i]
  p = Process(target=power_func, args=(num,key_p,key_n, que, i))
  p.start()   # prints "[42, None, 'hello']"


values = sorted((que.get() for _ in range(len(nums_d))), key=lambda r: r[0])

In [ ]:
for i, v in enumerate(values):
  nums_test[i] = v[1]

In [ ]:
nums_test

In [ ]:
import time

print(answer)
time.sleep(1)
i=0
print("".join(int_to_plain(dic_i_c, nums_test)))

## Easy way to decipher for poor vocabrary.

In [ ]:
def power_func(num,key_e,key_n, que, itr):
    bi = str(format(key_e,"b"))
    res = 1
    for i in range(len(bi)):
        res = (res*res) %key_n
        if bi[i] == "1":
            res = (res*num) %key_n
    que.put((itr, res))


que = Queue()
keys = dic_i_c.keys()
for i, num in enumerate(keys):
  p = Process(target=power_func, args=(num,key_e,key_n, que, i))
  p.start()

values = sorted((que.get() for _ in range(len(keys))), key=lambda r: r[0])

In [ ]:
#dic_i_c_all

In [ ]:
dic_i_c_all = dict()
for i, v in enumerate(values):
  dic_i_c_all[v[1]] = dic_i_c[i]

In [ ]:
for num in nums_d:
  print(dic_i_c_all[num], sep="", end="")